In [1]:
import numpy as np
import pandas as pd
import time,os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [2]:
data_path = '../data/'
op_train_sorted_file = 'op_train_sorted.csv' 
tran_train_sorted_file = 'tran_train_sorted.csv' 
tag_train_sorted_file = 'tag_train_sorted.csv' 

In [3]:
is_preprocessed = False
if not os.path.exists(data_path + op_train_sorted_file) or not os.path.exists(data_path + tran_train_sorted_file) or not os.path.exists(data_path + tag_train_sorted_file):
    print('[info]:start read from new train data...')
    is_preprocessed = False
    print('[info]:start read from op_train...')
    op_train = pd.read_csv('../data/operation_train_new.csv')
    print('[info]:start read from tran_train...')
    tran_train = pd.read_csv('../data/transaction_train_new.csv')
    print('[info]:start read from tag_train...')
    tag_train = pd.read_csv('../data/tag_train_new.csv')
else:
    print('[info]:start read from sorted data...')
    is_preprocessed = True
    print('[info]:start read from op_train...')
    op_train = pd.read_csv('../data/op_train_sorted.csv').drop('Unnamed: 0', axis=1)
    print('[info]:start read from tran_train...')
    tran_train = pd.read_csv('../data/tran_train_sorted.csv').drop('Unnamed: 0', axis=1)
    print('[info]:start read from tag_train...')
    tag_train = pd.read_csv('../data/tag_train_sorted.csv').drop('Unnamed: 0', axis=1)

[info]:start read from sorted data...
[info]:start read from op_train...
[info]:start read from tran_train...
[info]:start read from tag_train...


/home/alexliu/software/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1.数据预处理

In [4]:
def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
if not is_preprocessed:
    # 处理时间字符串
    op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
    op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

    tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
    tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))
    
    # 根据UID进行排序 再根据timestamp进行排序 
    op_train = op_train.sort_values(by=['UID', 'timestamp'],ascending=True).reset_index(drop=True)
    tran_train = tran_train.sort_values(by=['UID', 'timestamp'],ascending=True).reset_index(drop=True)
    tag_train = tag_train.sort_values(by=['UID'], ascending=True).reset_index(drop=True)

    op_train.to_csv(data_path + op_train_sorted_file)
    tran_train.to_csv(data_path + tran_train_sorted_file)
    tag_train.to_csv(data_path + tag_train_sorted_file)

In [6]:
# 去重
op_train_drop = op_train.drop_duplicates(inplace=True)
tran_train_drop = tran_train.drop_duplicates(inplace=True)
# 分组
op_train_gb = op_train.groupby('UID', as_index=False)
tran_train_gb = tran_train.groupby('UID', as_index=False)
#  获取op和tran各自的uid
op_train_uids = [uid for uid, item in op_train_gb.groups.items()]
tran_train_uids = [uid for uid, item in tran_train_gb.groups.items()]
# 获取op和tran各自的tag
op_tag = tag_train[tag_train['UID'].isin(op_train['UID'])]
tran_tag = tag_train[tag_train['UID'].isin(tran_train['UID'])]

#### 1.1处理缺失值

In [7]:
# op需要进行LabelEncoder的特征
op_columns = list(op_train.columns)
op_le_obj_fts = ['mode', 'os', 'version',
                  'device1', 'device2', 'device_code1', 'device_code2', 'device_code3',
                  'mac1', 'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub']
op_numtype_fts = ['success']

tran_columns = list(tran_train.columns)
tran_le_obj_fts = ['amt_src1', 'merchant',
                   'code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
                   'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1', 
                   'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2',
                   'market_code', 'ip1_sub']
tran_numtype_fts = ['channel', 'trans_amt', 'trans_type2', 'market_type']

In [8]:
def get_nan_counts(gb_count, ft_columns):
    hasnans_features_cnts = []
    for ft in ft_columns:
        cnts = gb_count[ft].value_counts()
        value = cnts[cnts.index == 0].values
        if len(value):
            hasnans_features_cnts.append((cnts.name,value[0]))
    return hasnans_features_cnts
        
def find_invalid_feature(gb_count, ft_columns):
    invalid_features = []
    for ft in ft_columns:
        cnts = gb_count[ft].value_counts()
        # 寻找值为0的统计数
        value = cnts[cnts.index == 0].values
        if len(value):
            if value[0] / gb_count.shape[0] > 0.5:
                print(cnts.name, value[0] / gb_count.shape[0])
                invalid_features.append(cnts.name)
    return invalid_features

def remove_list_item(src_l, rm_l):
    assert type(src_l) is list
    assert type(rm_l) is list
    
    for i in rm_l:
        if i in src_l:
            src_l.remove(i)
    return src_l

def preprocess(train_data, train_columns, le_obj_fts, numtype_fts):
    print('[info]: start fill nans...')
    for ft in numtype_fts:
        train_data[ft].fillna(-1, inplace=True)
        train_gb = train_data.groupby('UID', as_index=False)

    # 填补缺失值
    train_data = train_gb.ffill()
    train_gb = train_data.groupby('UID', as_index=False)
    train_data = train_gb.bfill()
    train_gb = train_data.groupby('UID', as_index=False)

    # 在填补基础上计数，去除nan值占一半以上的值
    print('[info]: start remove invalid features...')
    invalid_features = find_invalid_feature(train_gb.count(), train_columns)
    train_columns = remove_list_item(train_columns, invalid_features)
    le_obj_fts = remove_list_item(le_obj_fts, invalid_features)
    train_data.drop(invalid_features, axis='columns', inplace=True)

    # 填补剩余的缺失值
    print('[info]: start handle left nans...')
    op_hasnans_features_cnts = get_nan_counts(train_gb.count(), train_columns)
    for ft_cnts in op_hasnans_features_cnts:
        if train_data[ft_cnts[0]].hasnans:
            train_data[ft_cnts[0]].fillna('-1', inplace=True)
    train_gb = train_data.groupby('UID', as_index=False)
    print('[info]: handle nans finished.')

    # 对非数值型标签进行编码
    print('[info]: start label encoding...')
    le = LabelEncoder()
    for feature in le_obj_fts:
        try:
            print('[info]: %r label encoding...' %feature)
            train_data[feature] = le.fit_transform(train_data[feature])
        except TypeError as e:
            print(e)
    train_gb = train_data.groupby('UID', as_index=False)
    print('[info]: label encoding finished.')
    return train_data, train_gb, train_columns, le_obj_fts

def get_tag(uid):
    return tag_train[tag_train['UID'] == uid]

def get_op(uid):
    return op_train[op_train['UID'] == uid]

def get_tran(uid):
    return tran_train[tran_train['UID'] == uid]

def get_value_counts(uid, train_data):
    assert type(train_data) is pd.DataFrame
    for c in list(train_data.columns):
        print('[%r]'%c)
        print(train_data[train_data['UID']==uid][c].value_counts())
        print('====')

In [ ]:
op_train, op_train_gb, op_columns, op_le_obj_fts = preprocess(op_train, op_columns, op_le_obj_fts, op_numtype_fts)
#### tran的mac1探索

In [ ]:
for feature in op_le_obj_fts:
    try:
        print('[info]: %r value 0 replaced by -1 ...' %feature)
        op_train[feature].replace(0, -1, inplace=True)
    except TypeError as e:
        print(e)
op_train_gb = op_train.groupby('UID', as_index=False)

In [60]:
# 将op数据中的作假者与正常人分离开
op_train_fake_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 1])].groupby('UID', as_index=False)
op_train_normal_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 0])].groupby('UID', as_index=False)

In [ ]:
op_discv_fts = ['os', 'version', 'device1', 'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1']
tran_discv_fts = ['code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
                   'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1', 
                   'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2']

#### 典型羊毛党的特征

<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

### op

|字段名|解释|构造特征思路|
|-|-|-|
|day|操作日期|day_cnts，求day的总数|
|||top_appear_day，操作次数最多的day|
|mode|操作类型|op_times_per2min，每2分钟的操作次数|
| ||op_topcnts_oneday， 单天中操作的次数|
| ||mode_max，同一操作类型的最大次数| 
| ||mode_rank1，排名第一的操作类型| 
|success|操作状态|suc_rate,成功率|
|device_code1|安卓操作设备唯一标识1|dev_code_frq可以与code2、code3合并，统一为device_code，但是无法区分设备与电脑，可以考虑将电脑ip作为设备号添加进来，以减少Nan值，设备更换的频次 = 设备更换的次数/天数|
|device_code2|安卓操作设备唯一标识2||
|device_code3|苹果操作设备唯一标识1||
|ip1|操作设备的ip地址|ip_diff_oneday_top,可以与ip2合并，统计一天中的变化的最大次数|
|||ip_diff_cnt，统计变化次数|
|||ip_diff_frq，统计变化频次|
|ip2|操作电脑的ip地址||
|ip1_sub|操作设备的ip的前三位地址|ipsub_in_diffUID可以与ip2_sub合并，ip_sub出现在不同UID的次数|
|ip2_sub|操作电脑的ip的前三位地址||
|mac1|操作设备的mac地址|mac1_in_diffUID, mac1出现在不同UID的次数|
|device1|操作设备的名称|dev_name_frq, 将非nan的ip2_sub合入至该列，统计不同设备名称的频次 = 操作设备的名称变化总次数/总的操作次数|
|device2|操作设备的类型|dev_type_frq, 将非nan的ip2_sub合入至该列，统计不同设备名称的频次 = 操作设备的名称变化总次数/总的操作次数|
|mac2|wifi的mac地址|useless|
|geo_code|经纬度|useless|
|wifi|wifi名称|useless|
|os|操作系统|useless|
|version|软件版本|useless|

### tran

|字段名|解释|构造特征思路|
|-|-|-|
|channel|操作平台|channel_top_frq，出现次数最多的channel的频次|
|||channel_top，出现次数最多的channel|
|day|操作日期|day_cnts，求day的总数|
|||day_appear_top，操作次数最多的day|
|trans_amt|交易金额|tran_amt_frq，出现次数最多的金额的频次=出现次数最多的资金的总次数/总天数|
|||tran_amt_top，出现次数最多的交易金额|
|||tran_topcnts_oneday， 单天中交易的最多次数|
|amt_src1|资金类型|amt_src1_frq，资金类型变化的频率=资金类型变化的次数/总天数|
|||amt_src1_type_top，资金类型出现的最多类型|
|||amt_src1_type_cnt，出现最多的资金类型的个数|
|amt_src2|资金类型|amt_src2_frq，资金类型变化的频率=资金类型变化的次数/总天数|
|||amt_src2_type_top，资金类型出现的最多类型|
|||amt_src2_type_cnt，出现最多的资金类型的个数|
|merchant|商户标识，即商户编码|merchant_frq，商户标识变化的频率=商户标识变化的次数/总天数|
|||merchant_type_top，出现次数最多的商户标识类型|
|||merchant_type_cnt，商户标识类型总数|
|code1|商户标识，即商户子门店编码|code1_type_cnt，商户子门店类型总数|
|||code1_type_top，出现最多的商户子门店|
|trans_type1|交易类型，如消费退款等|trans_type1_top_cnt，出现次数最多的交易类型的次数|
|||trans_type1_top_frq，出现次数最多的交易类型的频次|
|||trans_type1_top，出现次数最多的交易类型|
|trans_type2|交易类型，如线上线下等|trans_type2_top_cnt，出现次数最多的交易类型的次数|
|||trans_type2_top_frq，出现次数最多的交易类型的频次|
|||trans_type2_top，出现次数最多的交易类型|
|acc_id1|用户账户号码|acc_id1_top_cnt,出现次数最多的交易类型的次数|
|||acc_id1_top_frq,出现次数最多的交易类型的频次|
|||acc_id1_top,出现次数最多的交易类型|
|device_code1|安卓操作设备唯一标识1|dev_code_frq可以与code2、code3合并，统一为device_code，但是无法区分设备与电脑，设备更换的频次 = 设备更换的次数/天数|
|device_code2|安卓操作设备唯一标识2||
|device_code3|苹果操作设备唯一标识1||
|device1|操作设备的名称|dev_name_frq，统计不同设备名称的频次 = 操作设备的名称变化总次数/总的操作次数|
|device2|操作设备的类型|dev_type_frq，统计不同设备名称的频次 = 操作设备的名称变化总次数/总的操作次数|
|mac1|操作设备的mac地址|mac1_in_diffUID, mac1出现在不同UID的次数|
|ip1|操作设备的ip地址|ip_diff_oneday_max,统计一天中的变化的最大次数|
|||ip_diff_cnt,统计所有统计中的变化次数|
|bal|账户余额|bal_top，出现最多的账户余额|
|||bal_top_cnt，出现最多的账户余额次数|
|||bal_ascend，上升的次数|
|||bal_ascend_max，上升时候增大的最大数|
|||bal_descend，上升的次数|
|||bal_descend_max，上升时候增大的最大数|
|market_code|营销活动编号|market_code_top，出现最多次数的营销编号|
|||market_code_top_cnt，出现最多营销编号的次数|
|||market_code_top_frq，出现最多营销编号的频次|
|ip1_sub|操作设备的ip的前三位地址|ip1_sub_in_diffUID，出现在不同UID的次数|
|acc_id2|转账时转出账户号码|useless|
|acc_id3|转账时转入账户号码|useless|
|market_type|营销活动标识|useless|
|code2|商户终端标识|useless|
|geo_code|经纬度|useless|
|os|操作系统|useless|
|version|软件版本|useless|

In [ ]:
op_train['ip_sub'] = op_train['ip1_sub'].replace(np.nan, '') + op_train['ip2_sub'].replace(np.nan, '')

In [ ]:
op_tag = pd.merge(op_train, tag_train, on='UID', how='left').replace(0, np.nan)
op_tag = op_tag.groupby('UID').count().drop(op_le_obj_fts, axis=1).drop(['success', 'time', 'timestamp'], axis=1)
op_tag = op_tag.reset_index()
op_tag['Tag'] = op_tag['Tag']/op_tag['day']
op_tag.drop('day',axis=1, inplace=True)
mode_max = []
mode_std = []
mode_rank1 = []
mode_count = []
for uid in list(op_train_gb.indices.keys()):
    uid_mode = op_train_gb.get_group(uid)['mode']
    mode_std.append(uid_mode.value_counts().std())    
    mode_max.append(uid_mode.value_counts().values[0])    
    mode_rank1.append(uid_mode.value_counts().index[0])
    mode_count.append(uid_mode.count())
op_tag['mode_max_rate'] = np.array(mode_max)/np.array(mode_count)*np.array(mode_std)
op_tag['mode_rank1'] = np.array(mode_rank1)

In [ ]:
# 查看排第一的操作类型个数和排名第一的操作类型的关系
le = LabelEncoder()
op_tag['mode_rank1'] = le.fit_transform(op_tag['mode_rank1'])
op_tag_fake = op_tag[op_tag['Tag'] == 0]
op_tag_normal = op_tag[op_tag['Tag'] == 1]
plt.xlim(-1,100)
plt.scatter(op_tag_normal['mode_max_rate'],op_tag_normal['mode_rank1'])

In [ ]:
new_fake_index = np.random.choice(op_tag_fake.index, size=op_tag_normal.shape[0])
new_fake_index
plt.xlim(-1,100)
op_part_fake = op_tag_fake.ix[pd.Index(new_fake_index)]
plt.scatter(op_part_fake['mode_max_rate'], op_part_fake['mode_rank1'])

In [ ]:
op_tag1 = op_tag[op_tag['Tag'] == 1]
op_tag0 = op_tag[op_tag['Tag'] == 0]
# op_tag_tag1_sorted = op_tag1['mode_std'].value_counts().sort_index(ascending=False)
# op_tag_tag0_sorted = op_tag0['mode_std'].value_counts().sort_index(ascending=False)

op_tag_tag1_sorted = op_tag1['mode_max'].value_counts().sort_index(ascending=False)
op_tag_tag0_sorted = op_tag0['mode_max'].value_counts().sort_index(ascending=False)

print('tag1:',op_tag_tag1_sorted[op_tag_tag1_sorted.index>50].count())
print('tag0:',op_tag_tag0_sorted[op_tag_tag0_sorted.index>50].count())

In [ ]:
x = op_tag_tag0_sorted.values
y = op_tag_tag0_sorted.index
plt.scatter(x, y)

In [ ]:
x = op_tag_tag1_sorted.values
y = op_tag_tag1_sorted.index
plt.scatter(x, y)

In [ ]:
op_tag_tag1_sorted.shape

In [ ]:
op_tag_tag0_sorted.shape

In [ ]:
op_tag_gb_sorted_tag0

In [ ]:
op_train_gb.get_group(17520)['mode'].value_counts().std()

In [ ]:
op_train[op_train['UID']==17520]['mode'].value_counts().std()

#### tran的mac1探索

In [40]:
tran_train[tran_train['UID']==10001].T

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
UID,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001
channel,140,140,102,102,102,102,102,102,102,102,102,102,102,102,102,140,140,102
day,2,2,3,4,7,7,7,8,11,11,12,12,13,13,13,30,30,30
time,2018-08-02 08:31:13,2018-08-02 08:31:13,2018-08-03 07:18:17,2018-08-04 22:30:51,2018-08-07 07:30:48,2018-08-07 07:34:35,2018-08-07 16:12:52,2018-08-08 07:49:23,2018-08-11 06:56:56,2018-08-11 07:02:09,2018-08-12 12:38:22,2018-08-12 12:38:22,2018-08-13 15:04:10,2018-08-13 15:04:10,2018-08-13 15:04:10,2018-08-30 07:09:48,2018-08-30 07:09:48,2018-08-30 15:38:09
trans_amt,9287,154,15186,15186,10293,81648,13628,15186,15186,108831,1385,12968,81648,1323,8254,6977,1377,10293
amt_src1,4d7831c6f695ab19,c5fc631370cabc0d,155c9e1c32bd0fa2,155c9e1c32bd0fa2,155c9e1c32bd0fa2,9451ef3c5a0d6807,155c9e1c32bd0fa2,155c9e1c32bd0fa2,155c9e1c32bd0fa2,9451ef3c5a0d6807,acdbdb842ac20f1e,155c9e1c32bd0fa2,155c9e1c32bd0fa2,acdbdb842ac20f1e,acdbdb842ac20f1e,4d7831c6f695ab19,c5fc631370cabc0d,155c9e1c32bd0fa2
merchant,f3237791ad270d6b,f3237791ad270d6b,a18cb3d84b9ec04c,a18cb3d84b9ec04c,a18cb3d84b9ec04c,3f6d3d0f42519ea4,3bd5cf7c40962299,a18cb3d84b9ec04c,a18cb3d84b9ec04c,3f6d3d0f42519ea4,d1136ca5db98c376,d1136ca5db98c376,d8babe2d19fa0c08,d8babe2d19fa0c08,d8babe2d19fa0c08,fd1e5bf59a1ee156,fd1e5bf59a1ee156,a18cb3d84b9ec04c
code1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
code2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans_type1,c2f2023d279665b2,c2f2023d279665b2,6d55c54c8b1056fb,6d55c54c8b1056fb,6d55c54c8b1056fb,6d55c54c8b1056fb,e0d7b8768da99dd4,6d55c54c8b1056fb,6d55c54c8b1056fb,6d55c54c8b1056fb,26bcf43a19df14c8,6d55c54c8b1056fb,6d55c54c8b1056fb,26bcf43a19df14c8,26bcf43a19df14c8,c2f2023d279665b2,c2f2023d279665b2,6d55c54c8b1056fb


In [64]:
op_train['mac1'].value_counts(0)

a8dc52f65085212e    48345
9251892b01b1a219     1096
3cbe2465ea11779b      978
90fc81ad8f40de4b      935
1ba06934bdd0299d      855
2b45ee1c3e9b1515      830
6b4f85615745676c      827
b1f92d4324c1ee16      800
e666a605ffc1171b      767
7127f1ad946b500a      687
7c8e00d275261110      667
36b57b355b216dab      648
e5f3425dc1d39040      636
84166db5b8202f36      628
94cc042c87ee134b      617
bf082474a2f515a6      548
610e4eaa21e4d52c      521
dd56b7aaa25e4da1      508
47359b8f8ace4198      501
cd17b7b823603aa8      473
63a10608a2c8426b      456
6cc26e4ed089a5e3      412
aca1e020465b752c      404
9eddf90faa75257b      391
16ea6aca0c06150a      363
627f86c85de1aa38      331
852fc46c3f0aff92      317
806392e223fb11e6      312
65ea75cf787a04c9      296
31771a5416acb922      295
                    ...  
1a4b01d5c0c24c51        1
0bb338344de8dae5        1
9e085b5ba69b8ed4        1
534de22256b4d7d9        1
2441f8382e6f1534        1
90e321b220a2a460        1
7469e0d457f9bf82        1
84a249f84891

In [57]:
t_mac1_gb = tran_tag.loc[:,['mac1', 'Tag']].groupby('mac1')

/home/alexliu/software/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [ ]:
tran_tag[tran_tag['mac1'].isin(t_mac1_des['Tag'][t_mac1_des['Tag']['std']>0].index)]['UID'].count()

In [43]:
t_mac1_des['Tag'][t_mac1_des['Tag']['std']>0].index

KeyError: 'Tag'

In [17]:
print(t_mac1_gb['Tag'])

In [63]:
fake_keys

[10000,
 10019,
 10021,
 10041,
 10058,
 10062,
 10064,
 10076,
 10097,
 10102,
 10105,
 10123,
 10165,
 10173,
 10185,
 10192,
 10201,
 10213,
 10217,
 10226,
 10252,
 10259,
 10262,
 10272,
 10293,
 10295,
 10303,
 10316,
 10339,
 10347,
 10351,
 10361,
 10367,
 10373,
 10374,
 10376,
 10378,
 10381,
 10390,
 10406,
 10417,
 10418,
 10426,
 10431,
 10437,
 10443,
 10448,
 10466,
 10471,
 10475,
 10484,
 10490,
 10496,
 10501,
 10503,
 10506,
 10507,
 10511,
 10513,
 10514,
 10537,
 10576,
 10594,
 10601,
 10611,
 10620,
 10643,
 10666,
 10678,
 10681,
 10685,
 10688,
 10692,
 10697,
 10721,
 10723,
 10725,
 10757,
 10767,
 10782,
 10783,
 10802,
 10816,
 10826,
 10843,
 10852,
 10854,
 10869,
 10872,
 10892,
 10921,
 10926,
 10937,
 10940,
 10943,
 10944,
 10955,
 10956,
 10957,
 10972,
 10976,
 10979,
 10988,
 10993,
 10995,
 10996,
 11006,
 11040,
 11052,
 11060,
 11063,
 11064,
 11077,
 11087,
 11121,
 11124,
 11130,
 11147,
 11149,
 11160,
 11171,
 11178,
 11185,
 11193,
 11201,


In [ ]:
for fk in fake_keys:
    print(op_train_fake_gp.get_group(fk)['mac1'].value_counts())

In [ ]:
tran_tag = pd.merge(tran_train, tag_train, on='UID', how='left')

In [ ]:
tran_tag

In [ ]:
tran_train_fake.count()

In [ ]:
tran_train_normal.count()

In [ ]:
fake_keys=[]
for key, item in op_train_fake_gp.indices.items():
    fake_keys.append(key)
    
normal_keys=[]
for key, item in op_train_normal_gp.indices.items():
    normal_keys.append(key)

In [ ]:
type(op_train_fake_gp.get_group(10000))
df_10000 = op_train_fake_gp.get_group(10000)
df_10000['mode'].value_counts()

In [ ]:
op_train['mode'].value_counts(dropna=False)

In [62]:
fake_keys=[]
for key, item in op_train_fake_gp.indices.items():
    fake_keys.append(key)

#### 1.2 tran处理

In [ ]:
# tran需要进行LabelEncoder的特征
tran_columns = ['UID', 'channel', 'day', 'time', 'trans_amt', 'amt_src1', 'merchant',
       'code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
       'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1',
       'bal', 'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2',
       'market_code', 'market_type', 'ip1_sub', 'timestamp']
tran_operator_features = []
tran_user_features = []